In [82]:
import copy
import datetime
import numpy as np
import os
import pandas as pd
import quandl

quandl.ApiConfig.api_key = "8DrzzNxwBzDzh_1jE6Dj"

y_hat_frbny = None

class Constant (object):
    TS = "time_series"
    DT = "data_table"
    DATA_DIR = "nowcast_inputs"
    OUTPUT_DIR = "nowcast_benchmarks"

QUANDL_DATA = {
    "WTI_DeptOfEnergy": {
        "refresh": False,
        "code": "EIA/PET_RWTC_D",
        "format": Constant.TS,
        "data": None
    },
    "AAPL_Zacks": {
        "refresh": False,
        "source": "ZACKS/FC",
        "ticker": "AAPL",
        "format": Constant.DT,
        "data": None
    },
    "SHORT_TERM_NAIRU_FedReserve": {
        "refresh": False,
        "code": "FRED/NROUST",
        "format": Constant.TS,
        "data": None
    },
    "CCY_AND_DEPOS_WorldBank": {
        "refresh": False,
        "code": "WPSD/USA_DP_DOD_DLCD_CR_GG_CD",
        "format": Constant.TS,
        "data": None
    },
    "CURR_ACC_BAL_WorldBank": {
        "refresh": False,
        "code": "WGEP/USA_BNCABFUNDCD_",
        "format": Constant.TS,
        "data": None
    },
    "UNEMPLOYMENT_FedReserve": {
        "refresh": False,
        "code": "FRED/M08K4BUSM148NNBR",
        "format": Constant.TS,
        "data": None
    },
    "BIZ_ACTIVITY_FedReserve": {
        "refresh": False,
        "code": "FRED/M12007USM511SNBR",
        "format": Constant.TS,
        "data": None
    },
    "20Y_YIELD_FedReserve": {
        "refresh": False,
        "code": "FRED/M13058USM156NNBR",
        "format": Constant.TS,
        "data": None
    },
    "NUM_INCORPORATIONS_FedReserve": {
        "refresh": False,
        "code": "FRED/M10090USM479NNBR",
        "format": Constant.TS,
        "data": None
    },
    "EMP_COMPENSATION_FedReserve": {
        "refresh": False,
        "code": "FRED/Q08358USQ027SNBR",
        "format": Constant.TS,
        "data": None
    },
    "FARM_INCOME_FedReserve": {
        "refresh": False,
        "code": "FRED/Q0876BUSQ027SNBR",
        "format": Constant.TS,
        "data": None
    },
    "PERSONAL_SAVING_FedReserve": {
        "refresh": False,
        "code": "FRED/Q1091BUSQ027SNBR",
        "format": Constant.TS,
        "data": None
    },
    ##
    "M1_FedReserve": {
        "refresh": False,
        "code": "FRED/MYAGM1USM052N",
        "format": Constant.TS,
        "data": None
    },
    "M2_FedReserve": {
        "refresh": False,
        "code": "FRED/MYAGM2USM052N",
        "format": Constant.TS,
        "data": None
    },
    "PREM_GAS_PX_FedReserve": {
        "refresh": False,
        "code": "FRED/GASPRMCOVM",
        "format": Constant.TS,
        "data": None
    },
    "PORTFOLIO_ASSETS_FedReserve": {
        "refresh": False,
        "code": "FRED/IEAAPIN",
        "format": Constant.TS,
        "data": None
    },
    "YLD_CURVE_FedReserve": {
        "refresh": False,
        "code": "FED/SVENPY",
        "format": Constant.TS,
        "data": None
    },
    "TECH_SPEND_FC_FedNY": {
        "refresh": False,
        "code": "FRBNY/TSFDINA",
        "format": Constant.TS,
        "data": None
    },
    "CAPEX_FC_FedNY": {
        "refresh": True,
        "code": "FRBNY/CEFDINA",
        "format": Constant.TS,
        "data": None
    },
    "EMPL_PVT_SVC_ADP": {
        "refresh": False,
        "code": "ADP/EMPL_SERVICE_PR",
        "format": Constant.TS,
        "data": None
    },
    "EMPL_NONFARM_ADP": {
        "refresh": False,
        "code": "ADP/EMPL_NONFARM_PRI",
        "format": Constant.TS,
        "data": None
    },
    "EMPL_GOODS_PROD_ADP": {
        "refresh": False,
        "code": "ADP/EMPL_GOODS_PROD",
        "format": Constant.TS,
        "data": None
    },
    "CAPEX_FC_FedNY": {
        "refresh": False,
        "code": "FRBNY/CEFDINA",
        "format": Constant.TS,
        "data": None
    },
    "QTRLY_GDP_PCT_CHG_Bea": {
        "refresh": False,
        "code": "BEA/NIPA_1_1_1_Q",
        "format": Constant.TS,
        "data": None
    }
}


In [83]:
def get_quandl_data ():
    for dc in QUANDL_DATA:
        if not QUANDL_DATA [dc]["refresh"]:
            continue
            
        ### sn: to get a numpy array, use the below. Don't worry about optimizing at this pt.
        ### quandl.get (QUANDL_DATA [dc] ["code"], returns = "numpy")
        
        if QUANDL_DATA [dc]["format"] == Constant.TS:
            QUANDL_DATA [dc] ["data"] = quandl.get (QUANDL_DATA [dc] ["code"])
            save_quandl_data_to_file (dc)
            #print (QUANDL_DATA [dc] ["data"])
        elif QUANDL_DATA [dc]["format"] == Constant.DT:
            QUANDL_DATA [dc] ["data"] = quandl.get_table (QUANDL_DATA [dc] ["source"], ticker=QUANDL_DATA [dc] ["ticker"])
            save_quandl_data_to_file (dc)
            #print (QUANDL_DATA [dc] ["data"])
        else:
            None

In [84]:
def save_quandl_data_to_file (dc):
    df = QUANDL_DATA [dc] ["data"]
    if isinstance (df, pd.DataFrame):
        file_name = Constant.DATA_DIR + "/" + dc + ".csv"
        print ("Saving dataframe to file " + file_name)
        df.to_csv (file_name, encoding='utf-8')
    

In [85]:
#if isinstance (QUANDL_DATA ["AAPL_Zacks"] ["data"], pd.DataFrame):
#    type (QUANDL_DATA ["AAPL_Zacks"] ["data"])

In [86]:
def load_data (refresh=False):
    global y_hat_frbny
    if refresh:
        get_quandl_data ()
    else:
        for dc in QUANDL_DATA:
            file_name = Constant.DATA_DIR + "/" + dc + ".csv"
            if os.path.isfile (file_name):
                QUANDL_DATA [dc] ["data"] = pd.read_csv (file_name)
          
    # sn: needs manual updates - nowcasts not avaialable in feed format
    y_hat_frbny = pd.read_csv (Constant.DATA_DIR + "/ny_fed.csv")
    

In [87]:
load_data (refresh=False)
#y_hat_frbny.tail (2)


In [88]:
nonfarm_df = QUANDL_DATA ["EMPL_NONFARM_ADP"] ["data"]
frbny_nowcast_df = y_hat_frbny
act_gdp_df = QUANDL_DATA ["QTRLY_GDP_PCT_CHG_Bea"] ["data"]
#act_gdp_df.tail (2)

In [89]:
#nonfarm_df ["Month"] [0]
#y_hat_frbny.tail (2)
nonfarm_df.tail(5)

,Month,1-19,20-49,1-49,50-499,500+,500-999,1000+
147,2017-04-30,31185.385497,20004.813552,51190.199050,44572.914511,28164.010312,8412.759968,19751.250344
148,2017-05-31,31232.189653,20020.458932,51252.648585,44683.298474,28224.666157,8444.816270,19779.849887
149,2017-06-30,31259.475306,20052.471538,51311.946844,44763.216890,28277.660309,8461.745058,19815.915251
150,2017-07-31,31284.442875,20076.487437,51360.930312,44847.879288,28345.226622,8485.091498,19860.135123
151,2017-08-31,31302.177864,20106.629514,51408.807378,44921.992017,28459.866774,8507.227306,19952.639468


In [102]:
def convert_yyyymmdd_str_to_qtr (dt_str):
    return dt_str [:4] + "Q" + str (int(np.floor((int (dt_str [5:7]) - 1)/3)) + 1)
    
def get_quarter_key (dt, qtr):
    if qtr*3 >= dt.month:
        return str (dt.year) + "Q" + str (qtr)
    else:
        return str (dt.year + 1) + "Q" + str (qtr)

def get_months_to_quarter (dt, qtr):
    if dt.month > qtr * 3:
        return (12 - dt.month + qtr*3 + 1)
    else:
        return (qtr*3 - dt.month + 1)

def mark_quarter (df, quarter):
    res_df = copy.deepcopy (df)
    #if quarter == 1:
    res_df ["TargetQuarter"] = res_df ['Date'].apply (get_quarter_key, args=(quarter,))
    res_df ["MonthsToTarget"] = res_df ['Date'].apply (get_months_to_quarter, args=(quarter,))
    return res_df
    
def featurize_monthly_absolute_figures (df, label_input_col, fmt="%Y-%m-%d"):
    df ["Date"] = df [label_input_col].apply (datetime.datetime.strptime, args=(fmt,))
    #x_df = pd.Dataframe
    #for i in range (1, 5):
    updated_df = mark_quarter (df, quarter=1)
    updated_df = updated_df.append (mark_quarter (df, quarter=2))
    updated_df = updated_df.append (mark_quarter (df, quarter=3))
    updated_df = updated_df.append (mark_quarter (df, quarter=4))
    return updated_df
    #df ["TargetPeriod"] = df [label_input_col]
    #df ["TargetPeriod"] = df [label_input_col].apply (convert_yyyymmdd_str_to_qtr)


In [103]:
updated_nonfarm_df = featurize_monthly_absolute_figures (nonfarm_df, "Month")
updated_nonfarm_df ['Total'] = updated_nonfarm_df ['1-49'] + updated_nonfarm_df ['50-499'] + updated_nonfarm_df ['500+']
#updated_nonfarm_df [['Date','TargetPeriod', 'MonthsToTarget', 'Total']].tail(50)
updated_nonfarm_df [updated_nonfarm_df['Month']=='2017-01-31']
#updated_nonfarm_df [['Month', 'MonthsToTarget']].tail (10)
#updated_nonfarm_df.shape
#nonfarm_df.shape
#type (nonfarm_df ['Date'][0])

,Month,1-19,20-49,1-49,50-499,500+,500-999,1000+,Date,TargetQuarter,MonthsToTarget,Total
144,2017-01-31,31057.993898,19878.039194,50936.033092,44284.823,28054.591742,8342.762328,19711.829414,2017-01-31,2017Q1,3,123275.447833
144,2017-01-31,31057.993898,19878.039194,50936.033092,44284.823,28054.591742,8342.762328,19711.829414,2017-01-31,2017Q2,6,123275.447833
144,2017-01-31,31057.993898,19878.039194,50936.033092,44284.823,28054.591742,8342.762328,19711.829414,2017-01-31,2017Q3,9,123275.447833
144,2017-01-31,31057.993898,19878.039194,50936.033092,44284.823,28054.591742,8342.762328,19711.829414,2017-01-31,2017Q4,12,123275.447833


In [106]:
updated_nonfarm_df [updated_nonfarm_df ['TargetQuarter'] == '2017Q2']

,Month,1-19,20-49,1-49,50-499,500+,500-999,1000+,Date,TargetQuarter,MonthsToTarget,Total
138,2016-07-31,30935.080038,19723.460718,50658.540756,43867.727689,27634.159054,8276.979113,19357.179941,2016-07-31,2017Q2,12,122160.427500
139,2016-08-31,30961.324340,19750.112916,50711.437256,43907.635990,27681.918454,8286.724026,19395.194428,2016-08-31,2017Q2,11,122300.991700
140,2016-09-30,30995.015065,19769.855426,50764.870490,43989.822838,27763.285472,8291.542198,19471.743273,2016-09-30,2017Q2,10,122517.978800
141,2016-10-31,30987.951877,19776.650323,50764.602199,44005.162254,27809.884647,8290.329664,19519.554983,2016-10-31,2017Q2,9,122579.649100
142,2016-11-30,31006.699399,19806.430013,50813.129411,44097.439918,27895.490470,8309.335533,19586.154937,2016-11-30,2017Q2,8,122806.059800
143,2016-12-31,31014.779590,19839.279667,50854.059257,44181.710194,27971.278749,8324.734737,19646.544012,2016-12-31,2017Q2,7,123007.048200
144,2017-01-31,31057.993898,19878.039194,50936.033092,44284.823000,28054.591742,8342.762328,19711.829414,2017-01-31,2017Q2,6,123275.447833
145,2017-02-28,31102.659821,19924.757580,51027.417401,44379.135357,28117.460521,8369.491088,19747.969433,2017-02-28,2017Q2,5,123524.013279
146,2017-03-31,31165.992255,19986.952416,51152.944670,44480.218760,28145.862943,8376.955597,19768.907346,2017-03-31,2017Q2,4,123779.026374
147,2017-04-30,31185.385497,20004.813552,51190.199050,44572.914511,28164.010312,8412.759968,19751.250344,2017-04-30,2017Q2,3,123927.123874


In [ ]:
X_df = 

In [113]:
X_df = updated_nonfarm_df.pivot (index='TargetQuarter', columns='MonthsToTarget', values='Total')
for i in range (1,13):
    X_df.rename (columns= {i: str(i) + "_mthPrior"}, inplace=True) 
X_df.tail (50)

Exception: Data must be 1-dimensional

In [ ]:
def featurize_monthly_absolute_figures_x (df, quarters):
    
    None
    
df1 = pd.DataFrame ([[1,11],[2,22]], columns=list('AB'))
df2 = pd.DataFrame ([[3,33],[4,44]], columns=list('AB'))
df3 = df1.append (df2)
df3

In [ ]:
nonfarm_df.tail(5)